In [7]:
import requests
import json
import pandas as pd
import re

In [25]:
draft_database = pd.read_csv('NHL_Drafts.csv')

In [26]:
draft_database

,Year,Round,Overall_Pick,Player_Name,Nationality
0,2015,1,1,Connor McDavid (C),Canada
1,2015,1,2,Jack Eichel (C),United States
2,2015,1,3,Dylan Strome (C),Canada
3,2015,1,4,Mitch Marner (C/RW),Canada
4,2015,1,5,Noah Hanifin (D),United States
...,...,...,...,...,...
1068,2019,7,213,McKade Webster (LW),United States
1069,2019,7,214,Dustin Wolf (G),United States
1070,2019,7,215,Arvid Costmar (C),Sweden
1071,2019,7,216,Massimo Rizzo (C),Canada


In [27]:
draft_database['Full_Name'] = draft_database['Player_Name'].apply(lambda x: re.sub(" *\\(.*", "", x))
draft_database['Position'] = draft_database['Player_Name'].apply(lambda x: re.search(r'\((.*?)\)',x).group(1))
del draft_database['Player_Name']

In [28]:
draft_database

,Year,Round,Overall_Pick,Nationality,Full_Name,Position
0,2015,1,1,Canada,Connor McDavid,C
1,2015,1,2,United States,Jack Eichel,C
2,2015,1,3,Canada,Dylan Strome,C
3,2015,1,4,Canada,Mitch Marner,C/RW
4,2015,1,5,United States,Noah Hanifin,D
...,...,...,...,...,...,...
1068,2019,7,213,United States,McKade Webster,LW
1069,2019,7,214,United States,Dustin Wolf,G
1070,2019,7,215,Sweden,Arvid Costmar,C
1071,2019,7,216,Canada,Massimo Rizzo,C


In [92]:
draft_database[(draft_database['Round'] == 6) & (draft_database['Position'] != 'G')]
# draft_database.drop_duplicates()

,Year,Round,Overall_Pick,Nationality,Full_Name,Position
151,2015,6,152,Canada,Giorgio Estephan,C
153,2015,6,154,United States,John Marino,D
154,2015,6,155,Canada,Stephen Desrocher,D
155,2015,6,156,Canada,Jake Massie,D
156,2015,6,157,Canada,Brett Seney,LW
...,...,...,...,...,...,...
1037,2019,6,182,Canada,Quinn Schmiemann,D
1038,2019,6,183,Canada,Blake Murray,C
1039,2019,6,184,Finland,Santeri Hatakka,D
1040,2019,6,185,Finland,Matias Mantykivi,C


In [ ]:
#Draft Picks By Round NOT GOALIE
# 1: 150
# 2: 144
# 3: 128
# 4: 141
# 5: 134
# 6: 131
# 7: 131

In [52]:
def return_a_player_stats(a_name):
    try:
        base_url = 'https://suggest.svc.nhl.com/svc/suggest/v1/minplayers/'
        # first_name = a_name.split()[0]
        # last_name = a_name.split()[1]
        num_to_return = '1'
        # first_name + '%20' + last_name 

        full_url = base_url + a_name + '/' + num_to_return
        response = requests.get(full_url)
        suggestions = json.loads(response.content)['suggestions'][0]
        player_info = str.split(suggestions, "|")
        player_id = player_info[0]
        url = 'https://statsapi.web.nhl.com/api/v1/people/' + player_id + '/stats/?stats=yearByYear'
        response = requests.get(url)
        content = json.loads(response.content)['stats']
        splits = content[0]['splits']

        df_splits = (pd.json_normalize(splits, sep = "_" )
                .query('league_name == "National Hockey League"')
        )
        df_splits['Full_Name'] = a_name

        return df_splits
    except IndexError:
        return 5

In [61]:
return_a_player_stats('Rasmus Dahlin')

,season,sequenceNumber,stat_assists,stat_goals,stat_pim,stat_games,stat_penaltyMinutes,stat_points,team_name,team_link,...,stat_gameWinningGoals,stat_overTimeGoals,stat_shortHandedGoals,stat_shortHandedPoints,stat_shortHandedTimeOnIce,stat_blocked,stat_shifts,team_id,league_id,Full_Name
25,20182019,1,35,9,34,82,34,44,Buffalo Sabres,/api/v1/teams/7,...,4.0,0.0,0.0,0.0,12:59,66.0,1905.0,7.0,133.0,Rasmus Dahlin
26,20192020,1,36,4,38,59,38,40,Buffalo Sabres,/api/v1/teams/7,...,1.0,0.0,0.0,0.0,05:44,51.0,1217.0,7.0,133.0,Rasmus Dahlin
27,20202021,1,18,5,26,56,26,23,Buffalo Sabres,/api/v1/teams/7,...,0.0,0.0,0.0,0.0,04:40,45.0,1242.0,7.0,133.0,Rasmus Dahlin
28,20212022,1,40,13,68,80,68,53,Buffalo Sabres,/api/v1/teams/7,...,3.0,1.0,0.0,1.0,72:58,91.0,2317.0,7.0,133.0,Rasmus Dahlin
31,20222023,1,49,14,76,58,76,63,Buffalo Sabres,/api/v1/teams/7,...,2.0,0.0,0.0,0.0,84:43,100.0,1784.0,7.0,133.0,Rasmus Dahlin


In [65]:
def create_a_stats_db():
    list_of_names = list(draft_database['Full_Name'])
    mega_db = pd.DataFrame()

    for name in list_of_names:
        temp_db = return_a_player_stats(name)
        if not isinstance(temp_db,int):
            mega_db = pd.concat([mega_db, temp_db], axis=0,ignore_index=True,sort=False) 

    return mega_db

In [66]:
return_a_player_stats('Juuso Parssinen')

,season,sequenceNumber,stat_assists,stat_goals,stat_pim,stat_games,stat_penaltyMinutes,stat_points,team_name,team_link,...,stat_gameWinningGoals,stat_overTimeGoals,stat_shortHandedGoals,stat_shortHandedPoints,stat_shortHandedTimeOnIce,stat_blocked,stat_shifts,team_id,league_id,Full_Name
16,20222023,1,18,5,15,42,15,23,Nashville Predators,/api/v1/teams/18,...,2.0,0.0,0.0,0.0,00:57,16.0,727.0,18.0,133.0,Juuso Parssinen


In [68]:
df = create_a_stats_db()

In [69]:
aggregated_df = pd.merge(draft_database,df,on='Full_Name')

In [70]:
# aggregated_df.to_csv('2015-2019_Draft_Players_all_seasons.csv',index=False)
aggregated_df

,Year,Round,Overall_Pick,Nationality,Full_Name,Position,season,sequenceNumber,stat_assists,stat_goals,...,stat_powerPlaySaves,stat_shortHandedSaves,stat_evenSaves,stat_shortHandedShots,stat_evenShots,stat_powerPlayShots,stat_gamesStarted,stat_powerPlaySavePercentage,stat_shortHandedSavePercentage,stat_evenStrengthSavePercentage
0,2015,1,1,Canada,Connor McDavid,C,20152016,1,32.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,Canada,Connor McDavid,C,20162017,1,70.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,Canada,Connor McDavid,C,20172018,1,67.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,Canada,Connor McDavid,C,20182019,1,75.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,Canada,Connor McDavid,C,20192020,1,63.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,2019,6,184,Finland,Santeri Hatakka,D,20212022,1,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1614,2019,7,201,Canada,Rafael Harvey-Pinard,LW,20212022,1,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1615,2019,7,201,Canada,Rafael Harvey-Pinard,LW,20222023,1,4.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1616,2019,7,203,Finland,Valtteri Puustinen,RW,20212022,1,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
#Now I must trim to only include 3 seasons from Draft Date
aggregated_df['season_played'] = aggregated_df.apply(lambda x: int(x['season'][4:]) - int(x['Year']),axis=1)
#If season_played is greater than 3 we should ignore

In [72]:
aggregated_df = aggregated_df[(aggregated_df['season_played'] <= 3) & (aggregated_df['season_played'] >= 1)] 

In [74]:
goalies_only = aggregated_df[aggregated_df['Position'] == 'G']

In [75]:
goalies_only

,Year,Round,Overall_Pick,Nationality,Full_Name,Position,season,sequenceNumber,stat_assists,stat_goals,...,stat_shortHandedSaves,stat_evenSaves,stat_shortHandedShots,stat_evenShots,stat_powerPlayShots,stat_gamesStarted,stat_powerPlaySavePercentage,stat_shortHandedSavePercentage,stat_evenStrengthSavePercentage,season_played
368,2015,3,76,Canada,Adin Hill,G,20172018,1,NaN,NaN,...,2.0,106.0,2.0,118.0,9.0,4.0,77.777778,100.000000,89.830508,3
762,2016,2,48,Canada,Carter Hart,G,20182019,1,NaN,NaN,...,31.0,777.0,32.0,848.0,96.0,30.0,90.625000,96.875000,91.627358,3
1178,2017,3,64,Canada,Michael DiPietro,G,20182019,1,NaN,NaN,...,1.0,13.0,1.0,20.0,3.0,1.0,100.000000,100.000000,65.000000,2
1179,2017,3,64,Canada,Michael DiPietro,G,20192020,1,NaN,NaN,...,2.0,4.0,2.0,4.0,1.0,0.0,0.000000,100.000000,100.000000,3
1231,2017,5,129,Switzerland,Gilles Senn,G,20192020,1,NaN,NaN,...,0.0,30.0,0.0,34.0,7.0,1.0,100.000000,NaN,88.235294,3
1250,2017,7,194,Canada,Dylan Ferguson,G,20172018,1,NaN,NaN,...,0.0,1.0,0.0,1.0,1.0,0.0,0.000000,NaN,100.000000,1
1253,2017,7,199,United States,Cayden Primeau,G,20192020,1,NaN,NaN,...,2.0,52.0,3.0,55.0,14.0,2.0,92.857143,66.666667,94.545455,3
1448,2018,4,114,Russia,Ivan Prosvetov,G,20202021,1,NaN,NaN,...,3.0,36.0,3.0,42.0,6.0,1.0,50.000000,100.000000,85.714286,3
1469,2018,6,173,Finland,Veini Vehvilainen,G,20202021,1,NaN,NaN,...,1.0,1.0,2.0,1.0,1.0,0.0,100.000000,50.000000,100.000000,3
1509,2019,1,13,United States,Spencer Knight,G,20202021,1,NaN,NaN,...,3.0,81.0,3.0,87.0,9.0,3.0,77.777778,100.000000,93.103448,2


In [93]:
goalies_only.to_csv('Goalie_Stats.csv',index=False)

In [76]:
all_other_players = aggregated_df[aggregated_df['Position'] != 'G']
all_other_players = all_other_players[all_other_players['Full_Name'] != 'Sebastian Aho']
all_other_players = all_other_players[all_other_players['Full_Name'] != 'Ryan O\'Reilly']
all_other_players.drop_duplicates()

,Year,Round,Overall_Pick,Nationality,Full_Name,Position,season,sequenceNumber,stat_assists,stat_goals,...,stat_shortHandedSaves,stat_evenSaves,stat_shortHandedShots,stat_evenShots,stat_powerPlayShots,stat_gamesStarted,stat_powerPlaySavePercentage,stat_shortHandedSavePercentage,stat_evenStrengthSavePercentage,season_played
0,2015,1,1,Canada,Connor McDavid,C,20152016,1,32.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2015,1,1,Canada,Connor McDavid,C,20162017,1,70.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,2015,1,1,Canada,Connor McDavid,C,20172018,1,67.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
8,2015,1,2,United States,Jack Eichel,C,20152016,1,32.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,2015,1,2,United States,Jack Eichel,C,20162017,1,33.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,2019,4,98,Finland,Matias Maccelli,LW,20212022,1,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1609,2019,5,125,United States,Mark Kastelic,C,20212022,1,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1613,2019,6,184,Finland,Santeri Hatakka,D,20212022,1,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1614,2019,7,201,Canada,Rafael Harvey-Pinard,LW,20212022,1,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [77]:
all_other_players.columns

Index(['Year', 'Round', 'Overall_Pick', 'Nationality', 'Full_Name', 'Position',
       'season', 'sequenceNumber', 'stat_assists', 'stat_goals', 'stat_pim',
       'stat_games', 'stat_penaltyMinutes', 'stat_points', 'team_name',
       'team_link', 'league_name', 'league_link', 'team_id', 'league_id',
       'stat_powerPlayGoals', 'stat_gameWinningGoals', 'stat_shortHandedGoals',
       'stat_plusMinus', 'stat_shots', 'stat_timeOnIce', 'stat_hits',
       'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_evenTimeOnIce',
       'stat_faceOffPct', 'stat_shotPct', 'stat_overTimeGoals',
       'stat_shortHandedPoints', 'stat_shortHandedTimeOnIce', 'stat_blocked',
       'stat_shifts', 'stat_savePercentage', 'stat_goalAgainstAverage',
       'stat_shutouts', 'stat_ties', 'stat_wins', 'stat_losses',
       'stat_goalsAgainst', 'stat_shotsAgainst', 'stat_ot', 'stat_saves',
       'stat_powerPlaySaves', 'stat_shortHandedSaves', 'stat_evenSaves',
       'stat_shortHandedShots', 'stat_eve

In [78]:

a_list = ['stat_savePercentage', 'stat_goalAgainstAverage',
       'stat_shutouts', 'stat_ties', 'stat_wins', 'stat_losses',
       'stat_goalsAgainst', 'stat_shotsAgainst', 'stat_ot', 'stat_saves',
       'stat_powerPlaySaves', 'stat_shortHandedSaves', 'stat_evenSaves',
       'stat_shortHandedShots', 'stat_evenShots', 'stat_powerPlayShots',
       'stat_gamesStarted', 'stat_powerPlaySavePercentage',
       'stat_shortHandedSavePercentage', 'stat_evenStrengthSavePercentage']

for stat in a_list:
    del all_other_players[stat]

In [79]:
all_other_players.to_csv('2015-2019-3Years-SkaterStats.csv', index=False)

In [103]:
#Avalible Player Stats
# Index(['Year', 'Round', 'Overall_Pick', 'Nationality', 'Full_Name', 'Position',
#        'season', 'sequenceNumber', 'stat_assists', 'stat_goals', 'stat_pim',
#        'stat_games', 'stat_penaltyMinutes', 'stat_points', 'team_name',
#        'team_link', 'league_name', 'league_link', 'team_id', 'league_id',
#        'stat_powerPlayGoals', 'stat_gameWinningGoals', 'stat_shortHandedGoals',
#        'stat_plusMinus', 'stat_shots', 'stat_timeOnIce', 'stat_hits',
#        'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_evenTimeOnIce',
#        'stat_faceOffPct', 'stat_shotPct', 'stat_overTimeGoals',
#        'stat_shortHandedPoints', 'stat_shortHandedTimeOnIce', 'stat_blocked',
#        'stat_shifts', 'season_played'],
#       dtype='object')

Index(['Year', 'Round', 'Overall_Pick', 'Nationality', 'Full_Name', 'Position',
       'season', 'sequenceNumber', 'stat_assists', 'stat_goals', 'stat_pim',
       'stat_games', 'stat_penaltyMinutes', 'stat_points', 'team_name',
       'team_link', 'league_name', 'league_link', 'team_id', 'league_id',
       'stat_powerPlayGoals', 'stat_gameWinningGoals', 'stat_shortHandedGoals',
       'stat_plusMinus', 'stat_shots', 'stat_timeOnIce', 'stat_hits',
       'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_evenTimeOnIce',
       'stat_faceOffPct', 'stat_shotPct', 'stat_overTimeGoals',
       'stat_shortHandedPoints', 'stat_shortHandedTimeOnIce', 'stat_blocked',
       'stat_shifts', 'season_played'],
      dtype='object')

In [88]:
all_other_players.groupby(['Round'])['stat_points'].mean()

Round
1    20.443478
2     6.662162
3     7.300000
4     6.789474
5     6.090909
6     9.888889
7     6.600000
Name: stat_points, dtype: float64

In [93]:
# 1: 150
# 2: 144
# 3: 128
# 4: 141
# 5: 134
# 6: 131
# 7: 131
number_of_skater_seasons_round1 = 150 * 3
number_of_skater_seasons_round2 = 144 * 3
number_of_skater_seasons_round3 = 128 * 3
number_of_skater_seasons_round4 = 141 * 3
number_of_skater_seasons_round5 = 134 * 3
number_of_skater_seasons_round6 = 131 * 3
number_of_skater_seasons_round7 = 131 * 3



#Nunber of seasons that have stats
round1_seasons_played = all_other_players[all_other_players['Round']==1].shape[0]
round2_seasons_played = all_other_players[all_other_players['Round']==2].shape[0]
round3_seasons_played = all_other_players[all_other_players['Round']==3].shape[0]
round4_seasons_played = all_other_players[all_other_players['Round']==4].shape[0]
round5_seasons_played = all_other_players[all_other_players['Round']==5].shape[0]
round6_seasons_played = all_other_players[all_other_players['Round']==6].shape[0]
round7_seasons_played = all_other_players[all_other_players['Round']==7].shape[0]


#Points per season average by first 3 seasons in NHL Directly after draft year
round1_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[0]
round2_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[1]
round3_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[2]
round4_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[3]
round5_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[4]
round6_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[5]
round7_mean_points_per_season = all_other_players.groupby(['Round'])['stat_points'].mean().iloc[6]


In [97]:
#Average point production by a first round pick for their first 3 eligible NHL seasons

first_round_mean_points_all_included = (round1_seasons_played * round1_mean_points_per_season) / number_of_skater_seasons_round1

second_round_mean_points_all_included = (round2_seasons_played * round2_mean_points_per_season) / number_of_skater_seasons_round2

third_round_mean_points_all_included = (round3_seasons_played * round3_mean_points_per_season) / number_of_skater_seasons_round3

fourth_round_mean_points_all_included = (round4_seasons_played * round4_mean_points_per_season) / number_of_skater_seasons_round4

fifth_round_mean_points_all_included = (round5_seasons_played * round5_mean_points_per_season) / number_of_skater_seasons_round5

sixth_round_mean_points_all_included = (round6_seasons_played * round6_mean_points_per_season) / number_of_skater_seasons_round6

seventh_round_mean_points_all_included = (round7_seasons_played * round7_mean_points_per_season) / number_of_skater_seasons_round7



In [115]:
print(f''' Average Point Production Per Season by an NHL Draft Pick For First 3 Eligible Seasons By Round
Round 1: {round(first_round_mean_points_all_included, 2)}
Round 2: {round(second_round_mean_points_all_included, 2)}
Round 3: {round(third_round_mean_points_all_included, 2)}
Round 4: {round(fourth_round_mean_points_all_included, 2)}
Round 5: {round(fifth_round_mean_points_all_included, 2)}
Round 6: {round(sixth_round_mean_points_all_included, 2)}
Round 7: {round(seventh_round_mean_points_all_included, 2)}''')

 Average Point Production Per Season by an NHL Draft Pick For First 3 Eligible Seasons By Round
Round 1: 10.45
Round 2: 1.14
Round 3: 0.38
Round 4: 0.3
Round 5: 0.17
Round 6: 0.23
Round 7: 0.08


In [109]:
all_round_point_total = first_round_mean_points_all_included + second_round_mean_points_all_included + \
    third_round_mean_points_all_included + fourth_round_mean_points_all_included + \
        fifth_round_mean_points_all_included + sixth_round_mean_points_all_included + \
            second_round_mean_points_all_included 

In [110]:
all_round_point_total

13.809598939629087

In [116]:
print(f''' Percentage of Point Creation By Round by NHL Draft Picks for First 3 Eligible Seasons
Round 1: {round(first_round_mean_points_all_included / all_round_point_total, 3)}
Round 2: {round(second_round_mean_points_all_included / all_round_point_total, 3)}
Round 3: {round(third_round_mean_points_all_included / all_round_point_total,3)}
Round 4: {round(fourth_round_mean_points_all_included / all_round_point_total,3)}
Round 5: {round(fifth_round_mean_points_all_included / all_round_point_total,3)}
Round 6: {round(sixth_round_mean_points_all_included / all_round_point_total,3)}
Round 7: {round(seventh_round_mean_points_all_included / all_round_point_total,3)}''')

 Percentage of Point Creation By Round by NHL Draft Picks for First 3 Eligible Seasons
Round 1: 0.757
Round 2: 0.083
Round 3: 0.028
Round 4: 0.022
Round 5: 0.012
Round 6: 0.016
Round 7: 0.006
